In [46]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from keras.utils.np_utils import to_categorical
import tensorflow as tf
with h5py.File("data/full_dataset_vectors.h5", 'r') as h5:
    X_train, y_train = h5["X_train"][:], h5["y_train"][:]
    X_test, y_test = h5["X_test"][:], h5["y_test"][:]
    
# Translate data to color
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

def translate(x):
    xx = np.ndarray((x.shape[0], 4096, 3))
    for i in range(x.shape[0]):
        xx[i] = array_to_color(x[i])
        if i % 1000 == 0:
            print(i)
    # Free Memory
    del x

    return xx
X_train = translate(X_train).reshape(-1, 16, 16, 16, 3)
X_test  = translate(X_test).reshape(-1, 16, 16, 16, 3)

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
0
1000


In [ ]:
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = X_train.shape[0] // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#初始化偏置值
def bias_variable(shape):
    initial = tf.zeros(shape)+0.1
    return tf.Variable(initial)

#卷积层
def conv3d(x,w):
    return tf.nn.conv3d(x,w,strides=[1,1,1,1,1],padding='SAME')

#池化层
def maxpool(x):
    return tf.nn.max_pool3d(x,ksize=[1,2,2,2,1],strides=[1,2,2,2,1],padding='SAME')

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,16,16,16,3],name="x-input")
y=tf.placeholder(tf.float32,[None,10],name="y-input")
keep_prob = tf.placeholder(tf.float32)

#改变x的向量变成4D的结构
x_image = tf.reshape(x,[-1,16,16,16,3])

#对第一卷积层的filter和bias做初始化
w_conv1 = weight_variable([3,3,3,3,32])
b_conv1 = bias_variable([32])

#把x_image和filter做卷积，在加上bias，作用于relu激活函数
h_conv1 = tf.nn.relu(conv3d(x_image,w_conv1)+b_conv1)
h_pool1 = maxpool(h_conv1)

#对第二卷积层的filter和bias做初始化
w_conv2 = weight_variable([3,3,3,32,64])
b_conv2 = bias_variable([64])

#把第一层输出和filter做卷积，在加上bias，作用于relu函数
h_conv2 = tf.nn.relu(conv3d(h_pool1,w_conv2)+b_conv2)
h_pool2 = maxpool(h_conv2)

#把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,4*4*4*64])

#初始化第一个全连接层的权值和bias
w_fc1 = weight_variable([4*4*4*64,1024])
b_fc1 = bias_variable([1024])

#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
h_fcl_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层的权值和bias
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

#计算第二个全连接层的输出
prediction = tf.nn.softmax(tf.matmul(h_fc1,w_fc2)+b_fc2)

#交叉熵代价函数 loss指标表示模型的好壞
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
#计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1):
        for batch in range(n_batch):
            _,lo = sess.run([train_step,loss],feed_dict={x:X_train[batch*100:batch*100+99,:,:],y:y_train[batch*100:batch*100+99,:],keep_prob:0.7})

        testing_acc = sess.run(accuracy,feed_dict={x:X_test,y:y_test,keep_prob:1.0})
        training_acc = sess.run(accuracy,feed_dict={x:X_train,y:y_train,keep_prob:1.0})
        print("iter"+str(epoch)+",testing accuracy"+str(testing_acc)+",training accuracy"+str(training_acc)+",loss"+str(lo))

In [ ]:
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = X_train.shape[0] // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#初始化偏置值
def bias_variable(shape):
    initial = tf.zeros(shape)+0.1
    return tf.Variable(initial)

#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

#池化层
def maxpool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,4096],name="x-input")
y=tf.placeholder(tf.float32,[None,10],name="y-input")
keep_prob = tf.placeholder(tf.float32)

#改变x的向量变成4D的结构
x_image = tf.reshape(x,[-1,16,16,16])

#对第一卷积层的filter和bias做初始化
w_conv1 = weight_variable([3,3,16,32])
b_conv1 = bias_variable([32])

#把x_image和filter做卷积，在加上bias，作用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
h_pool1 = maxpool(h_conv1)

#对第二卷积层的filter和bias做初始化
w_conv2 = weight_variable([3,3,32,64])
b_conv2 = bias_variable([64])

#把第一层输出和filter做卷积，在加上bias，作用于relu函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2 = maxpool(h_conv2)

#把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,4*4*64])

#初始化第一个全连接层的权值和bias
w_fc1 = weight_variable([4*4*64,1024])
b_fc1 = bias_variable([1024])

#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
h_fcl_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层的权值和bias
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

#计算第二个全连接层的输出
prediction = tf.nn.softmax(tf.matmul(h_fc1,w_fc2)+b_fc2)

#交叉熵代价函数 loss指标表示模型的好壞
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
#计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(51):
        for batch in range(n_batch):
            _,lo = sess.run([train_step,loss],feed_dict={x:X_train[batch*100:batch*100+99,:],y:y_train[batch*100:batch*100+99,:],keep_prob:0.7})

        testing_acc = sess.run(accuracy,feed_dict={x:X_test,y:y_test,keep_prob:1.0})
        training_acc = sess.run(accuracy,feed_dict={x:X_train,y:y_train,keep_prob:1.0})
        print("iter"+str(epoch)+",testing accuracy"+str(testing_acc)+",training accuracy"+str(training_acc)+",loss"+str(lo))